In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import time
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp


device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

# CoordiNET architecture 1

    - Compute Distances through coordinates using the standard euclidian distance equation
    - Once we have the 729x729x1 distances matrix corresponding to the flatten 9x9x9 matrix, we do simultaneous row/column permutation in order to shuffle the array
    - We apply the noise before shuffling the 729x729x1 array. Because we reconstruct the noisy matrix and not the original one 
    - In the neural network, we conv the shuffled input and convTranspose it again -> It's an autoencoder, we try to reconstruct the original distance matrix from the shuffled one.
    
    - We have to define what kind of noise and transformation we want to add to the distance matrix:
        - Compute a 729x729x1 matrix of noize. If the noize get above a specified threshold there will be a specified transformation to this cell
        - Transformation must be pairwise, because we need to get back an Hermitian matrix. FIXED with mirrorDiagonally(mat, size)
        
If there is very high correlated cells, i.e. cells that have low distances between every other cells: we need to be able to create this kind of random cubes in order to simulate a real case


Try changing the distribution of the distances with rankGauss for example

# What to do next?

    DONE
    Try using a rank gauss before and after quite every step
    -> Why ? Because we are not looking to compute the exact value of each cells 
    We are trying to compute the rank values of each cells in order to get an idea of which cell ended up in which other cell

Try creating a neural layer where the network computes row permutations. Our problem is a permutation problem, then we must find the best way to compute those permutations

# Cube

In [ ]:
SIZE = 9
BATCH_SIZE= 64
STEPS_PER_EPOCH = 100

In [ ]:
class Cell():
    def __init__(self, coord, size):
        self.size = size
        self.coordinates = coord
        self.dist_mat = np.zeros(size**3)
        self.getDistMat(coord, size)

    def getDistMat(self, coord, size):
        iter_ = 0
        for k in range(size):
            for j in range(size):
                for i in range(size):
                    coord_ = np.array((i,j,k))
                    self.dist_mat[iter_] = np.linalg.norm(coord_ - coord)
                    iter_ += 1

In [ ]:
class Cube():
    def __init__(self, size):
        self.size = size
        self.cells = []
        self.getCells()
        self.dist_mat = np.zeros((size**3, size**3))
        self.getDistMat()
    
    def getCells(self):
        size=self.size
        for k in range(size):
            for j in range(size):
                for i in range(size):
                    coord = np.array((i,j,k))
                    self.cells.append(Cell(coord, size))
                    
    def getDistMat(self):
        cells = self.cells
        for i in range(self.size**3):
            self.dist_mat[i] = cells[i].dist_mat

# Dataset generator


In [ ]:
def rankRow(x):
    from scipy.special import erfinv
    N = x.shape[0]
    temp = x.argsort()
    rank_x = temp.argsort() / N
    rank_x -= rank_x.mean()
    rank_x *= 2
    efi_x = erfinv(rank_x)
    efi_x -= efi_x.mean()
    return efi_x

def rankArray(x):
    x_ = np.array(x)
    for i in range(x.shape[0]):
        x_[i] = rankRow(x_[i])
    return x_

def rankBatch(X):
    X_ = []
    for x in X:
        x = np.reshape(x, (SIZE**3, SIZE**3))
        X_.append(rankArray(x))
    return X #np.expand_dims(X,3)

In [ ]:
class DatasetGenerator():
    def __init__(self, size, rng_level=0, ds_size=100, batch_size=32):
        self.cube = Cube(size)
        self.size = self.cube.size
        self.rng_level = rng_level
        self.ds_size = ds_size
        self.population = np.arange(self.size**3).tolist()
        self.batch_size = batch_size
    
    def getGen(self):
        ds_size = self.ds_size
        batch_size = self.batch_size
        for i in range(ds_size):
            mat = np.array(self.cube.dist_mat)
            X,y = [],[]
            for n in range(batch_size):
                trans_mat = self.transformMat(mat)
                permut_mat = self.permutMat(trans_mat)
                X.append((rankArray(permut_mat)+1.474569062)/(2*1.474569062)), y.append(rankArray(trans_mat))
            #yields X, y (the shuffled transformed mat and the unshuffled transformed mat)
            X,y = np.array(X), np.array(y)
            yield np.expand_dims(X,3), np.expand_dims(y,3)
    
    def transformMat(self, mat_):
        population = self.population
        # Compute overcorrelated and undercorrelated transformation
        over_corr = int(self.size**3*random.uniform(0.001,0.05)) # between 0.1% and 5% of the features
        under_corr = int(self.size**3*random.uniform(0.05,0.3)) # between 5% and 30% of the features
        idx = random.sample(population, over_corr+under_corr)
        oc_idx = idx[:over_corr]
        oc_scalar = 1/(np.random.randint(low=10 ,high=100 ,size=over_corr)) #We divide the over correlated because they have low distances between everything
        uc_idx = idx[over_corr:]
        uc_scalar = np.random.randint(low=10 ,high=100 ,size=under_corr)
        for i in range(over_corr):
            mat_[oc_idx[i]] = mat_[oc_idx[i]]*oc_scalar[i]
        for i in range(under_corr):
            mat_[uc_idx[i]] = mat_[uc_idx[i]]*uc_scalar[i]
        rand_mat = 1 - (np.random.random((self.size**3,self.size**3))-0.5)*self.rng_level
        mat_ = mat_*rand_mat
        return self.mirrorDiagonally(mat_)
    
    def mirrorDiagonally(self,mat):
        for i in range(self.size):
            for j in range(self.size):
                mat[i,j] = mat[j,i]
        return mat

    def permutMat(self, mat):
        rng_permut = np.random.permutation(self.size**3)
        mat_ = mat[rng_permut, :]
        mat_ = mat_[:,rng_permut]
        return mat_

# Dataset Generator

In [ ]:
ds_gen = DatasetGenerator(SIZE, batch_size=BATCH_SIZE).getGen()

# Training Neural Network

In [ ]:
from tensorflow import keras

class VAE(keras.Model):
    def __init__(self, ds_gen, latent_dim=2):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
    
    
    def train(self,epochs=100, batch_size=BATCH_SIZE):
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
        for epoch in range(1, epochs+1):
            t = time.time()
            last_loss=0
            ds_gen = DatasetGenerator(SIZE, batch_size=BATCH_SIZE).getGen()
            for X, y in ds_gen:
                gradients, loss = self.compute_gradients(X,y)
                self.apply_gradients(optimizer, gradients, self.trainable_variables)
                last_loss = loss
            print('Epoch {}, Loss: {}, Time spent: {:.2f}'.format(epoch, last_loss, time.time() - t))
    
    def inferenceNet(self,x):
        x = keras.layers.InputLayer(input_shape=(SIZE**3,SIZE**3,1))(x)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(1024, activation='relu')(x)
        x = keras.layers.Dense(512, activation='relu')(x)
        x = keras.layers.Dense(self.latent_dim*2)(x)        
        return x
    
    def generativeNet(self,x):
        z = keras.layers.InputLayer(input_shape=(self.latent_dim))(x)
        z = keras.layers.Dense(512, activation='relu')(z)
        z = keras.layers.Dense(1024, activation='relu')(z)
        z = keras.layers.Dense(SIZE**3 * SIZE**3)(z)
        z = keras.layers.Reshape(target_shape=(SIZE**3,SIZE**3,1))(z)
        return z
    
    def encode(self,x):
        mean_logvar = self.inferenceNet(x)
        N = mean_logvar.shape[0]
        mean = tf.slice(mean_logvar, [0, 0], [N, self.latent_dim])
        logvar = tf.slice(mean_logvar, [0, self.latent_dim], [N, self.latent_dim])
        return mean, logvar
    
    def decode(self,z, apply_sigmoid=False):
        logits = self.generativeNet(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits
    
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps*tf.exp(logvar*.5) + mean
    
    def compute_gradients(self, x, y):
        with tf.GradientTape() as tape:
            loss = self.compute_loss(x, y)
        return tape.gradient(loss, self.trainable_variables), loss    
    
    def compute_loss(self, x,y):
        mean, logvar = self.encode(x)
        z = self.reparameterize(mean, logvar)
        x_logits = self.decode(z)
        x_logits = tf.cast(rankBatch(x_logits), 'float32')
        y = tf.cast(y, 'float32')
        loss = tf.keras.losses.MAE(y, x_logits)
        loss = tf.convert_to_tensor(np.sum(loss))
        return loss
    
    def apply_gradients(self, optimizer, gradients, variables):
        optimizer.apply_gradients(zip(gradients, variables))
        pass
        

model = VAE(ds_gen)
model.train()

# Sources

    - Image restoration: https://research.nvidia.com/sites/default/files/pubs/2017-03_Loss-Functions-for/NN_ImgProc.pdf